## More on SQL

Author: Greg Wray  
2024-MAR-04

### Set-up and build database
The steps below follow those in the notebook from last week (refer to that notebook for explanations). The only difference here is that we load `matplotlib` so that we can do some graphing with results in Python. 

In [ ]:
# import packages and extensions
import numpy as np
import pandas as pd
import matplotlib 
import duckdb
%load_ext sql

In [ ]:
# create an in-memory database
%sql duckdb://

In [ ]:
%%sql

-- create 7 tables and read in data for each from .csv files
    
DROP TABLE IF EXISTS orders;             
CREATE TABLE orders (                    
    order_ioc VARCHAR PRIMARY KEY,       
    seq SMALLINT NOT NULL,               
    familiar_order VARCHAR,              
    taxonomy VARCHAR                     
    );                                   
COPY orders FROM 's_orders.csv';         

DROP TABLE IF EXISTS families;
CREATE TABLE families(
    family_ioc VARCHAR PRIMARY KEY,
    seq SMALLINT NOT NULL,
    order_ioc VARCHAR NOT NULL,
    familiar_family VARCHAR,
    niche VARCHAR,
    taxonomy VARCHAR,
    num_gen SMALLINT NOT NULL,
    num_spp SMALLINT NOT NULL,
    num_spp_x SMALLINT NOT NULL,
    num_threat SMALLINT NOT NULL
    );
COPY families FROM 's_families.csv';

DROP TABLE IF EXISTS genera;
CREATE TABLE genera(
    genus_ioc VARCHAR PRIMARY KEY,
    seq SMALLINT NOT NULL,
    family_ioc VARCHAR NOT NULL,
    familiar_genus VARCHAR,
    taxonomy VARCHAR,
    num_spp SMALLINT NOT NULL
    );
COPY genera FROM 's_genera.csv';

DROP TABLE IF EXISTS species;
CREATE TABLE species(
    seq SMALLINT PRIMARY KEY,
    genus_ioc VARCHAR NOT NULL,
    species_ioc VARCHAR NOT NULL,
    num_spp SMALLINT NOT NULL,
    familiar_ioc VARCHAR,
    conservation VARCHAR,
    endemic VARCHAR
    );
COPY species FROM 's_species.csv';

DROP TABLE IF EXISTS observations;
CREATE TABLE observations(
    seq SMALLINT PRIMARY KEY,
    genus_ioc VARCHAR NOT NULL,
    species_ioc VARCHAR NOT NULL,
    subspecies_ioc VARCHAR NOT NULL,
    date_obs DATE,
    time_obs VARCHAR,
    location_name VARCHAR NOT NULL,
    trip_name VARCHAR NOT NULL,
    notes VARCHAR
    );
COPY observations FROM 's_observations.csv';

DROP TABLE IF EXISTS trips;
CREATE TABLE trips(
    trip_name VARCHAR PRIMARY KEY,
    start_date DATE NOT NULL,
    end_date DATE NOT NULL,
    notes VARCHAR
    );
COPY trips FROM 's_trips.csv';

DROP TABLE IF EXISTS locations;
CREATE TABLE locations(
    location_name VARCHAR PRIMARY KEY,
    province VARCHAR,
    country_name VARCHAR,
    bioregion_name VARCHAR,
    climate VARCHAR,
    protection VARCHAR,
    earliest DATE NOT NULL,
    latest DATE NOT NULL
    );
COPY locations FROM 's_locations.csv';

In [ ]:
# remove default maximum on number of records returned
%config SqlMagic.displaylimit = 0  

## Creating and modifying tables

YOUR TURN: Attempt to query the `trips` table. You should now get a catalog error stating that `trips` does not exist. You can easily re-generate the table by running the code block that creates all the tables. Go ahead and do that now before we move on to the next topic.

## Data integrity

YOUR TURN: Run a query to make sure that the update worked as expected. Now uncomment the part of the `CREATE TABLE` statement above and attempt to re-create the table and establish the foreign key constraint. Test the foreign key constraint by attempting to modify the name 'Kanha Kisli NP' in the `locations` table to an incorrect spelling. As another test, attempt to modify the value of `location_name` in any row by misspelling it or by entering a place that has never been visited (e.g., 'Grand Canyon NP').  

## Relational database design

YOUR TURN: Design a database to store the results of a set of camera-traps at a field site. Assume that there are 12 camera traps, each of which may capture from zero to many images per night, that the study lasts for 20 days, and that the study aims to record every case where any kind of mammal sets off the trap. As part of the study, each morning you and your team will examine the images captured the night before and enter the results into the database. Of greatest interest are the species imaged, but you also plan to capture relevant meta-data. For instance, you will have access to the field station's automated weather logging records. And you want the data to be useful for possible future studies, so the design should be question-agnostic.

Sketch out a set of tables and columns that could hold this information. Think about which columns need constraints and whethere any foreign key constraints would be helpful. Make sure your database is in third normal form.

Designing a database can seem daunting. A good way to start is to experiment by creating tables and loading some sample data. Write code to create the tables and columns for your database. Insert two or more rows of sample data into each table. Design a few test queries to get a feel for how you might retrieve data. Consider making changes and re-create tables as needed to implement them. Iterate through changes and testing. 

## Working with SQL within Python

YOUR TURN: Write a Python program that creates a graph showing the cumulative number of observations over every year from 1967 (the earliest observation) to 2023 (the most recent year with complete data). Use matplotlib to construct the graph. You can use either a line graph or a histogram. Note that you will need to fill in values for years for which there are no new observations. Once that's working, add the cumulative number of new species observed over the same interval.